# ライトニングネットワーク中心性分析

このノートブックでは、ビットコインのライトニングネットワークにおける中心性指標を分析します。

## 理論的背景

本分析は論文「A Centrality Analysis of the Lightning Network」 (2201.07746v1) に基づいています。

### 中心性指標

1. **媒介中心性 (Betweenness Centrality)**: ノードが他のノード間の最短経路にどれだけ頻繁に現れるか
   - 高中心性ノードは多くのルーティング経路に含まれる
   - ルーティング手数料を重みとして使用した重み付き最短経路で計算

2. **近接中心性 (Closeness Centrality)**: ノードから他のすべてのノードへの平均距離の逆数
   - ネットワーク全体へのアクセスしやすさを測定

3. **固有ベクトル中心性 (Eigenvector Centrality)**: 重要なノードに接続されているノードが重要とされる
   - ノードの影響力を測定

### ルーティング手数料の計算

論文に基づき、ルーティング手数料は以下の式で計算されます：

**手数料 = fee_base_msat + (transaction_size_msat × fee_proportional_millionths / 1,000,000)**

エッジの重みは手数料の逆数として設定され、手数料が低いほど重みが高くなります。

## 目的

どのノードとチャネルを開設したらルーティングがされやすくなるかを分析します。


## 0. 依存パッケージのインストール

**重要**: 初回実行時、またはパッケージが更新された場合は、このセルを実行してください。


In [18]:
# requirements.txtからすべてのパッケージをインストール
# 注意: インストール後はカーネルを再起動してください

import os
import sys
import subprocess

# プロジェクトルートを取得
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
requirements_path = os.path.join(project_root, 'requirements.txt')

if os.path.exists(requirements_path):
    print(f"requirements.txtを読み込みます: {requirements_path}")
    print("パッケージをインストールしています...")
    
    # subprocessを使用してpip installを実行
    # %pipマジックコマンドでは変数展開ができないため、subprocessを使用
    try:
        result = subprocess.run(
            [sys.executable, '-m', 'pip', 'install', '-r', requirements_path],
            capture_output=True,
            text=True,
            check=False
        )
        
        if result.returncode == 0:
            print(result.stdout)
            print("\n✓ パッケージのインストールが完了しました")
            print("⚠️  重要: カーネルを再起動してください（Kernel → Restart Kernel）")
        else:
            print("エラーが発生しました:")
            print(result.stderr)
            print("\n代替方法: 以下のコマンドを手動で実行してください")
            print(f"  %pip install -r {requirements_path}")
    except Exception as e:
        print(f"インストール中にエラーが発生しました: {e}")
        print("\n代替方法: 以下のコマンドを手動で実行してください")
        print(f"  %pip install -r {requirements_path}")
else:
    print(f"✗ requirements.txtが見つかりません: {requirements_path}")
    print(f"現在のディレクトリ: {os.getcwd()}")
    print(f"プロジェクトルート: {project_root}")


requirements.txtを読み込みます: c:\Users\taishi\OneDrive\ドキュメント\日本ビットコイン産業株式会社\Centrality\requirements.txt
パッケージをインストールしています...


Exception in thread Thread-8 (_readerthread):
Traceback (most recent call last):
  File "c:\Users\taishi\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "c:\Users\taishi\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\taishi\AppData\Local\Programs\Python\Python311\Lib\subprocess.py", line 1599, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
UnicodeDecodeError: 'cp932' codec can't decode byte 0x88 in position 182: illegal multibyte sequence


None

✓ パッケージのインストールが完了しました
⚠️  重要: カーネルを再起動してください（Kernel → Restart Kernel）


## 1. 環境設定とインポート


In [20]:
import sys
import os

# プロジェクトルートをパスに追加
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, project_root)

import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import logging
from pathlib import Path

# ロギング設定
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

# 日本語フォント設定（必要に応じて）
plt.rcParams['font.family'] = 'DejaVu Sans'
sns.set_style("whitegrid")

print("環境設定が完了しました")


環境設定が完了しました


## 2. データベース接続とデータ取得


In [22]:
import importlib
from src.database.connection import DatabaseConnection
from src.graph import builder
# モジュールを再インポート（キャッシュ問題を回避）
importlib.reload(builder)
from src.graph.builder import GraphBuilder
from datetime import datetime

# データベース接続
db = DatabaseConnection()

# グラフを構築
# 注意: days_backを指定することで、最新のデータのみを使用できます（推奨: 30-90日）
# Noneの場合は全期間のデータを使用します
graph_builder = GraphBuilder(db)
transaction_size_msat = 1_000_000_000  # 0.01 BTC（論文のデフォルト値）
days_back = 90  # 最新90日以内のデータのみを使用（妥当なグラフを構築するため）

print("グラフを構築しています...")
print(f"使用するデータ: 最新{days_back}日以内のアクティブなチャネルのみ")
print(f"トランザクションサイズ: {transaction_size_msat:,} msat (0.01 BTC)")

graph = graph_builder.build_graph(
    directed=False, 
    transaction_size_msat=transaction_size_msat,
    days_back=days_back
)

print(f"\n✓ グラフが構築されました:")
print(f"  ノード数: {graph.number_of_nodes():,}")
print(f"  エッジ数: {graph.number_of_edges():,}")
print(f"  連結成分数: {nx.number_connected_components(graph)}")
if graph.number_of_nodes() > 0:
    print(f"  平均次数: {sum(dict(graph.degree()).values()) / graph.number_of_nodes():.2f}")
    
    # グラフの妥当性を確認
    largest_component = max(nx.connected_components(graph), key=len)
    print(f"  最大連結成分のノード数: {len(largest_component):,}")
    print(f"  最大連結成分の割合: {len(largest_component) / graph.number_of_nodes() * 100:.1f}%")


2025-12-18 01:25:32,668 - src.database.connection - INFO - データベース接続プールを初期化しました
2025-12-18 01:25:32,669 - src.graph.builder - INFO - エッジデータを取得しています...
2025-12-18 01:25:32,670 - src.graph.builder - INFO - 最新90日以内のデータのみを取得します


グラフを構築しています...
使用するデータ: 最新90日以内のアクティブなチャネルのみ
トランザクションサイズ: 1,000,000,000 msat (0.01 BTC)


2025-12-18 01:26:18,784 - src.graph.builder - INFO - 45817個のエッジを取得しました
2025-12-18 01:26:18,797 - src.graph.builder - INFO - 最新のチャネル更新: 2025-12-13 23:56:58
2025-12-18 01:26:18,820 - src.graph.builder - INFO - ノードデータを取得しています...
2025-12-18 01:32:55,234 - src.graph.builder - INFO - 5314038個のノードを取得しました
2025-12-18 01:33:14,773 - src.graph.builder - INFO - グラフを構築しました（ノード数: 8724, エッジ数: 40506）
2025-12-18 01:33:14,775 - src.graph.builder - INFO - 重み付きグラフとして構築しました（トランザクションサイズ: 1,000,000,000 millisatoshi）



✓ グラフが構築されました:
  ノード数: 8,724
  エッジ数: 40,506
  連結成分数: 52
  平均次数: 9.29
  最大連結成分のノード数: 8,607
  最大連結成分の割合: 98.7%


## 3. 中心性計算


In [29]:
import importlib
import sys

# モジュールを強制的に再インポート（キャッシュ問題を回避）
modules_to_reload = [
    'src.centrality.calculator',
    'src.centrality.betweenness',
    'src.centrality.closeness',
    'src.centrality.eigenvector'
]

for module_name in modules_to_reload:
    if module_name in sys.modules:
        del sys.modules[module_name]
        print(f"キャッシュから削除: {module_name}")

# モジュールを再インポート
from src.centrality.calculator import CentralityCalculator

# 中心性計算器を初期化
calculator = CentralityCalculator()

# すべての中心性指標を計算
print("中心性計算を開始します...")
print(f"使用するトランザクションサイズ: {transaction_size_msat:,} msat (0.01 BTC)")
centrality_scores = calculator.calculate_all(graph)

print("\n計算が完了しました:")
for name, scores in centrality_scores.items():
    if scores:
        values = list(scores.values())
        top_node = max(scores.items(), key=lambda x: x[1])
        print(f"  {name}:")
        print(f"    ノード数: {len(scores)}")
        print(f"    平均: {np.mean(values):.6f}")
        print(f"    最大: {top_node[1]:.6f} (ノード: {top_node[0]})")
        print(f"    最小: {np.min(values):.6f}")


2025-12-18 02:15:40,765 - src.centrality.calculator - INFO - 媒介中心性の計算を開始します...
2025-12-18 02:15:40,789 - src.centrality.betweenness - INFO - 媒介中心性の計算を開始します（ノード数: 8724, エッジ数: 40506）
2025-12-18 02:15:40,921 - src.centrality.betweenness - INFO - グラフのweight属性を自動検出しました（ルーティング手数料ベース）


キャッシュから削除: src.centrality.calculator
キャッシュから削除: src.centrality.betweenness
キャッシュから削除: src.centrality.closeness
キャッシュから削除: src.centrality.eigenvector
中心性計算を開始します...
使用するトランザクションサイズ: 1,000,000,000 msat (0.01 BTC)


2025-12-18 03:27:22,534 - src.centrality.betweenness - INFO - 媒介中心性の計算が完了しました（ノード数: 8724）
2025-12-18 03:27:22,550 - src.centrality.betweenness - INFO -   平均: 0.000271, 最大: 0.265133, 最小: 0.000000
2025-12-18 03:27:22,551 - src.centrality.betweenness - INFO -   重み付き計算を使用しました（weight='weight'）
2025-12-18 03:27:22,552 - src.centrality.calculator - INFO - 近接中心性の計算を開始します...
2025-12-18 03:27:22,562 - src.centrality.closeness - INFO - 近接中心性の計算を開始します（ノード数: 8724, エッジ数: 40506）
2025-12-18 03:27:22,649 - src.centrality.closeness - WARNING - グラフは非連結です（連結成分数: 52）。wf_improved=Trueが推奨されます。
2025-12-18 03:27:22,734 - src.centrality.closeness - INFO - グラフのweight属性を距離として自動検出しました
2025-12-18 04:07:08,596 - src.centrality.closeness - INFO - 近接中心性の計算が完了しました（ノード数: 8724）
2025-12-18 04:07:08,599 - src.centrality.closeness - INFO -   平均: 0.288578, 最大: 0.475263, 最小: 0.000115
2025-12-18 04:07:08,599 - src.centrality.closeness - INFO -   重み付き計算を使用しました（distance='weight'）
2025-12-18 04:07:08,599 - src.centrality.closenes


計算が完了しました:
  betweenness:
    ノード数: 8724
    平均: 0.000271
    最大: 0.265133 (ノード: 03864ef025fde8fb587d989186ce6a4a186895ee44a926bfc370e2c366597a3f8f)
    最小: 0.000000
  closeness:
    ノード数: 8724
    平均: 0.288578
    最大: 0.475263 (ノード: 03864ef025fde8fb587d989186ce6a4a186895ee44a926bfc370e2c366597a3f8f)
    最小: 0.000115
  eigenvector:
    ノード数: 8724
    平均: 0.003554
    最大: 0.184848 (ノード: 03864ef025fde8fb587d989186ce6a4a186895ee44a926bfc370e2c366597a3f8f)
    最小: 0.000000


In [23]:
## 4. 中心性の可視化


In [30]:
from src.analysis.visualizer import GraphVisualizer

# 可視化器を初期化
visualizer = GraphVisualizer(output_dir='../results')

# 中心性分布を可視化
visualizer.plot_centrality_distribution(centrality_scores)

# 各中心性タイプでグラフを可視化（上位100ノード）
for centrality_type in ['betweenness', 'closeness', 'eigenvector']:
    if centrality_type in centrality_scores:
        visualizer.plot_graph_with_centrality(
            graph, 
            centrality_scores, 
            centrality_type=centrality_type,
            top_n=100
        )

# 上位ノードの比較
visualizer.plot_top_nodes_comparison(centrality_scores, top_n=20)

print("可視化が完了しました")


c:\Users\taishi\OneDrive\ドキュメント\日本ビットコイン産業株式会社\Centrality\src\analysis\visualizer.py:51: UserWarning: Glyph 20013 (\N{CJK UNIFIED IDEOGRAPH-4E2D}) missing from current font.
  plt.tight_layout()
c:\Users\taishi\OneDrive\ドキュメント\日本ビットコイン産業株式会社\Centrality\src\analysis\visualizer.py:51: UserWarning: Glyph 24515 (\N{CJK UNIFIED IDEOGRAPH-5FC3}) missing from current font.
  plt.tight_layout()
c:\Users\taishi\OneDrive\ドキュメント\日本ビットコイン産業株式会社\Centrality\src\analysis\visualizer.py:51: UserWarning: Glyph 24615 (\N{CJK UNIFIED IDEOGRAPH-6027}) missing from current font.
  plt.tight_layout()
c:\Users\taishi\OneDrive\ドキュメント\日本ビットコイン産業株式会社\Centrality\src\analysis\visualizer.py:51: UserWarning: Glyph 12473 (\N{KATAKANA LETTER SU}) missing from current font.
  plt.tight_layout()
c:\Users\taishi\OneDrive\ドキュメント\日本ビットコイン産業株式会社\Centrality\src\analysis\visualizer.py:51: UserWarning: Glyph 12467 (\N{KATAKANA LETTER KO}) missing from current font.
  plt.tight_layout()
c:\Users\taishi\OneDrive\ドキュメント\日本ビットコイン産

可視化が完了しました


## 6. ルーティング可能性分析


In [31]:
from src.analysis.analyzer import RoutingAnalyzer

# ルーティング分析器を初期化
analyzer = RoutingAnalyzer(graph, centrality_scores)

# 推奨ノードを取得（上位50ノード）
recommended_nodes = analyzer.recommend_nodes_for_channel(top_n=50)

print("推奨ノード（上位20）:")
print(recommended_nodes.head(20).to_string())

# CSVに保存
recommended_nodes.to_csv('../results/recommended_nodes.csv', index=False)
print("\n推奨ノードをCSVに保存しました")

# 推奨チャネルを取得（上位100チャネル）
top_nodes = recommended_nodes.head(100)['node_id'].tolist()
recommended_channels = analyzer.recommend_channels(
    candidate_nodes=top_nodes,
    top_n=100
)

print("\n推奨チャネル（上位20）:")
print(recommended_channels.head(20).to_string())

# CSVに保存
recommended_channels.to_csv('../results/recommended_channels.csv', index=False)
print("\n推奨チャネルをCSVに保存しました")


2025-12-18 11:44:40,442 - src.analysis.analyzer - INFO - 上位50ノードの推薦を完了しました


推奨ノード（上位20）:
                                                                 node_id  degree  betweenness_centrality  closeness_centrality  eigenvector_centrality  routing_potential
6299  03864ef025fde8fb587d989186ce6a4a186895ee44a926bfc370e2c366597a3f8f    1941                0.265133              0.475263                0.184848           0.312115
5566  03ccc570ec6aaff08d5435b3413f4b4af8175728a1ed244e4710121c8f5af6ea07    1064                0.221209              0.370719                0.006526           0.223125
102   02c953421bc7f07be6052920e46843d11e6d3ffc9986177c91f140d76c6ed3a3d4     916                0.192799              0.403755                0.011373           0.219801
171   03423790614f023e3c0cdaa654a3578e919947e4c3a14bf5044e7c787ebd11af1a     494                0.065621              0.467182                0.148289           0.202623
76    035e4ff418fc8b5554c5d9eea66396c227bd429a3251c8cbc711002ba215bfc226    1042                0.068356              0.428492           

2025-12-18 11:45:02,243 - src.analysis.analyzer - INFO - 上位100チャネルの推薦を完了しました



推奨チャネル（上位20）:
                                                                source                                                              target  existing_channel  source_routing_potential  target_routing_potential  combined_potential  shortest_path_length  shortest_paths_count
0   03864ef025fde8fb587d989186ce6a4a186895ee44a926bfc370e2c366597a3f8f  03ccc570ec6aaff08d5435b3413f4b4af8175728a1ed244e4710121c8f5af6ea07             False                  0.312115                  0.223125            0.267620                     2                    12
1   03864ef025fde8fb587d989186ce6a4a186895ee44a926bfc370e2c366597a3f8f  0217890e3aad8d35bc054f43acc00084b25229ecff0ab68debd82883ad65ee8266             False                  0.312115                  0.136157            0.224136                     2                   160
2   03ccc570ec6aaff08d5435b3413f4b4af8175728a1ed244e4710121c8f5af6ea07  035e4ff418fc8b5554c5d9eea66396c227bd429a3251c8cbc711002ba215bfc226             False          

## 7. 特徴量抽出（機械学習用）


In [ ]:
from src.ml.features import FeatureExtractor

# 特徴量抽出器を初期化
feature_extractor = FeatureExtractor(graph, centrality_scores)

# ノード特徴量を抽出
node_features_df = feature_extractor.create_node_dataframe()
print(f"ノード特徴量: {node_features_df.shape}")
print(node_features_df.head())

# エッジ特徴量を抽出
edge_features_df = feature_extractor.create_edge_dataframe()
print(f"\nエッジ特徴量: {edge_features_df.shape}")
print(edge_features_df.head())

# CSVに保存
node_features_df.to_csv('../results/node_features.csv', index=False)
edge_features_df.to_csv('../results/edge_features.csv', index=False)
print("\n特徴量をCSVに保存しました")


In [ ]:
## 8. グラフニューラルネットワーク（GNN）による分析（オプション）

このセクションでは、DGLグラフへの変換と保存を行います。
AWS SageMakerでの訓練については、`docs/GNN_CENTRALITY_ANALYSIS_GUIDE.md`と`docs/AWS_SAGEMAKER_GUIDE.md`を参照してください。

### 8.1 DGLグラフへの変換と保存


In [ ]:
import dgl
import torch
import json

# NetworkXグラフをDGLグラフに変換
print("DGLグラフに変換しています...")
node_list = list(graph.nodes())  # ノードの順序を保持
dgl_graph = dgl.from_networkx(graph, node_attrs=None, edge_attrs=None)

# ノード特徴量の準備
num_nodes = dgl_graph.number_of_nodes()

# 基本特徴量（次数）
degrees = torch.tensor([graph.degree(n) for n in node_list], dtype=torch.float32)
features = degrees.unsqueeze(1)

# 中心性指標を取得（ノードの順序を保持）
betweenness = torch.tensor([centrality_scores['betweenness'].get(n, 0.0) 
                            for n in node_list], dtype=torch.float32)
closeness = torch.tensor([centrality_scores['closeness'].get(n, 0.0) 
                         for n in node_list], dtype=torch.float32)
eigenvector = torch.tensor([centrality_scores['eigenvector'].get(n, 0.0) 
                           for n in node_list], dtype=torch.float32)

# 中心性も特徴量として使用
features = torch.cat([features, 
                      betweenness.unsqueeze(1),
                      closeness.unsqueeze(1),
                      eigenvector.unsqueeze(1)], dim=1)

# 特徴量の正規化
feature_mean = features.mean(dim=0, keepdim=True)
feature_std = features.std(dim=0, keepdim=True) + 1e-8
features_normalized = (features - feature_mean) / feature_std

dgl_graph.ndata['feat'] = features_normalized

# 中心性指標を個別に保存（分析用）
dgl_graph.ndata['betweenness'] = betweenness
dgl_graph.ndata['closeness'] = closeness
dgl_graph.ndata['eigenvector'] = eigenvector

# ラベルの準備（ルーティング可能性）
routing_potential = (betweenness * 0.5 + closeness * 0.3 + eigenvector * 0.2)
dgl_graph.ndata['label'] = routing_potential.unsqueeze(1)

# グラフを保存
dgl.save_graphs('../results/lightning_graph.bin', [dgl_graph])

# メタデータの保存
metadata = {
    'num_nodes': dgl_graph.number_of_nodes(),
    'num_edges': dgl_graph.number_of_edges(),
    'feature_dim': features.shape[1],
    'transaction_size_msat': transaction_size_msat
}

with open('../results/graph_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"DGLグラフ情報:")
print(f"  ノード数: {dgl_graph.number_of_nodes()}")
print(f"  エッジ数: {dgl_graph.number_of_edges()}")
print(f"  特徴量次元: {features_normalized.shape[1]}")
print(f"  ラベル形状: {dgl_graph.ndata['label'].shape}")
print(f"\nグラフデータを保存しました: ../results/lightning_graph.bin")


## 9. 結果をS3に保存（オプション）

**推奨構成**: ローカル（Cursor/Jupyter）で実行し、結果をS3に保存するハイブリッドアプローチ

**利点**:
- **コスト効率**: SageMaker Notebook Instanceの料金がかからない
- **開発速度**: 即座に実行可能、デバッグが容易
- **永続化**: 結果をS3に保存することで、データの永続化と共有が可能


In [ ]:
# 分析結果をS3に保存（オプション）
# ローカル（Cursor/Jupyter）で実行した結果をS3に保存して永続化

import boto3
import os
from pathlib import Path
from datetime import datetime
import yaml

# 設定ファイルからS3バケット情報を取得
try:
    config_path = os.path.join(project_root, 'config', 'config.yaml')
    with open(config_path, 'r', encoding='utf-8') as f:
        config = yaml.safe_load(f)
    bucket_name = config.get('aws', {}).get('s3', {}).get('bucket')
    s3_prefix = config.get('aws', {}).get('s3', {}).get('prefix', 'lightning-network-analysis/')
except Exception as e:
    print(f"設定ファイルの読み込みエラー: {e}")
    bucket_name = None
    s3_prefix = 'lightning-network-analysis/'

if bucket_name:
    # S3クライアントの作成
    s3 = boto3.client('s3', region_name='ap-northeast-1')
    
    # タイムスタンプ付きのディレクトリ名を作成
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    s3_results_prefix = f"{s3_prefix}results/{timestamp}/"
    
    # ローカルのresultsディレクトリ
    local_results_dir = Path(project_root) / 'results'
    
    print(f"S3バケット: {bucket_name}")
    print(f"S3プレフィックス: {s3_results_prefix}")
    print(f"\n結果をS3にアップロードしています...")
    
    uploaded_files = []
    
    # resultsディレクトリ内のすべてのファイルをアップロード
    if local_results_dir.exists():
        for file_path in local_results_dir.rglob('*'):
            if file_path.is_file():
                relative_path = file_path.relative_to(local_results_dir)
                s3_key = f"{s3_results_prefix}{relative_path.as_posix()}"
                
                try:
                    s3.upload_file(str(file_path), bucket_name, s3_key)
                    uploaded_files.append(s3_key)
                    print(f"  ✓ {file_path.name} → s3://{bucket_name}/{s3_key}")
                except Exception as e:
                    print(f"  ✗ {file_path.name} のアップロードに失敗: {e}")
    
    if uploaded_files:
        print(f"\n✓ {len(uploaded_files)}個のファイルをS3にアップロードしました")
        print(f"S3パス: s3://{bucket_name}/{s3_results_prefix}")
    else:
        print("\n⚠️  アップロードするファイルが見つかりませんでした")
else:
    print("⚠️  S3バケット名が設定されていません")
    print("config/config.yamlのaws.s3.bucketを設定してください")


## 10. 結果のまとめ


In [ ]:
print("=== 分析結果のまとめ ===\n")

print(f"グラフ統計:")
print(f"  ノード数: {graph.number_of_nodes()}")
print(f"  エッジ数: {graph.number_of_edges()}")
print(f"  平均次数: {sum(dict(graph.degree()).values()) / graph.number_of_nodes():.2f}")
print(f"  連結成分数: {nx.number_connected_components(graph)}")

print(f"\n中心性統計:")
for name, scores in centrality_scores.items():
    if scores:
        values = list(scores.values())
        top_node = max(scores.items(), key=lambda x: x[1])
        print(f"  {name}:")
        print(f"    最大値ノード: {top_node[0]}")
        print(f"    最大値: {top_node[1]:.6f}")
        print(f"    平均値: {np.mean(values):.6f}")

if 'recommended_nodes' in locals():
    print(f"\n推奨ノード数: {len(recommended_nodes)}")
if 'recommended_channels' in locals():
    print(f"推奨チャネル数: {len(recommended_channels)}")

print("\n分析が完了しました。結果は results/ ディレクトリに保存されています。")


## 11. クリーンアップ


In [ ]:
# データベース接続を閉じる
db.close()
print("データベース接続を閉じました")
